In [1]:
#!/usr/bin/python3

import pypsa
import pandas as pd
import numpy as np
import sys
from pypsa.opt import Constraint as Con

import pdb

!hostname

resi


In [2]:
pypsa.__file__

'/home/vres/py/PyPSA/pypsa/__init__.py'

In [3]:
pypsa.opf.extract_optimisation_results??

In [ ]:
def renewable_scenario_constraints(n,snapshots,limits=None,flexibility=0.1):
    carrier_constraints = ['hydro', 'nuclear', 'hydro ror']
    
    for constraint in limits.index:
        if constraint in carrier_constraints:
            if constraint == 'hydro':
                index = n.storage_units[n.storage_units['carrier']==constraint].index
                setattr(n.model,constraint+"_limit_low",Con(expr=sum(n.model.storage_p_nom[name] \
                                                          for name in index) >= limits[constraint] * (1.-flexibility)))
                setattr(n.model,constraint+"_limit_up",Con(expr=sum(n.model.storage_p_nom[name] \
                                                          for name in index) <= limits[constraint] * (1.+flexibility)))
            else:
                index = n.generators[n.generators['carrier']==constraint].index
                setattr(n.model,constraint+"_limit_low",Con(expr=sum(n.model.generator_p_nom[name] \
                                                          for name in index) >= limits[constraint] * (1.-flexibility)))
                setattr(n.model,constraint+"_limit_up",Con(expr=sum(n.model.generator_p_nom[name] \
                                                          for name in index) <= limits[constraint] * (1.+flexibility)))
    return

def renewable_set_constraints(network, snapshots):
    return renewable_scenario_constraints(network, snapshots, limits = limits, flexibility = flexibility)


model = 'fias-model'
data = 'era5-data'
name = 'vietnam'
fn = './vietnam3_storage.nc'

attribute = 'renewable'  # or 'devplan'
storage = True
year = 2030

scenarios = pd.read_csv('./scenario_limits.csv')
scenarios = scenarios.set_index(scenarios['scenario']).drop(columns='scenario').dropna(axis=1)
limits = scenarios.loc[year]
flexibility = 0.1

n = pypsa.Network(fn)

co2_limit = 1.84*87.84e6*0.05 #tons_per_citizen*citizens http://www.worldbank.org/ 2011 data
n.global_constraints = n.global_constraints.reindex(['co2_limit'])
n.global_constraints.type = 'primary_energy'
n.global_constraints.carrier_attribute = 'co2_emissions'
n.global_constraints.sense = '<='
n.global_constraints.constant = co2_limit #tons
n.global_constraints.mu = 0.0

#n.storage_units.p_nom_max = np.Inf
n.loads_t.p_set = n.loads_t.p_set * limits['load_increase']

if storage == True:
    scenario = '{}_{}_{}'.format(attribute, year, 'storage')
else:
    scenario = '{}_{}'.format(attribute, year)
n.lopf(extra_functionality = renewable_set_constraints,
       snapshots=n.snapshots[:8760],
       solver_name='gurobi',
       solver_options={"threads":20, "method":0, "crossover":0},
       formulation='kirchhoff', keep_files=False,
       solver_logfile='test.log')

n.name = name + '_' + scenario
n.export_to_csv_folder('/home/schlott/Vietnam/ReVietSys/test')

INFO:pypsa.io:Imported network vietnam3_storage.nc has buses, carriers, generators, lines, loads, storage_units
INFO:pypsa.pf:Slack bus for sub-network 0 is CongThanh
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.pf:Tree slack bus is PhoNoi with degree 4.
INFO:pypsa.pf:Slack bus for sub-network 0 is CongThanh
INFO:pypsa.pf:Slack bus for sub-network 0 is CongThanh
INFO:pypsa.opf:Solving model using gurobi


In [12]:
n.generators.p_nom_opt

name
CongThanh hard coal     None
DiLinh wind             None
DongAnh hard coal       None
DuyenHai hard coal      None
HiepHoa hard coal       None
LongPhu wind            None
MongDuong2 hard coal    None
NhaBe ocgt              None
NhaBe oil               None
NhoQuan hard coal       None
OMon bioenergy          None
OMon ocgt               None
OMon oil                None
PhoNoi hard coal        None
PhuLam oil              None
PhuMy hard coal         None
PhuMy ocgt              None
Pleiku2 bioenergy       None
Pleiku2 wind            None
QuangNinh hard coal     None
SongMay oil             None
ThangLongT hard coal    None
ThangLongT oil          None
ThanhMy hard coal       None
VinhTan hard coal       None
VinhTan wind            None
VungAng hard coal       None
CauBong wind            None
ChonThanh wind          None
CongThanh wind          None
                        ... 
DaNang hydro ror        None
DocSoi hydro ror        None
ThanhMy hydro ror       None
Pleiku hy

In [ ]:
n.model.generator_p.display()

In [13]:
n.model.generator_p_nom.display()

generator_p_nom : Size=414, Index=generator_p_nom_index
    Key                  : Lower  : Value : Upper         : Fixed : Stale : Domain
       CauBong bioenergy :    0.0 :  None :           inf : False :  True : NonNegativeReals
       CauBong hard coal :    0.0 :  None :           inf : False :  True : NonNegativeReals
       CauBong hydro ror :    0.0 :  None : 1.65199087837 : False :  True : NonNegativeReals
         CauBong nuclear :    0.0 :  None :           inf : False :  True : NonNegativeReals
            CauBong ocgt :    0.0 :  None :           inf : False :  True : NonNegativeReals
             CauBong oil :    0.0 :  None :           inf : False :  True : NonNegativeReals
       CauBong perpetuum :    0.0 :  None :           inf : False :  True : NonNegativeReals
           CauBong solar :    0.0 :  None : 73001.9223763 : False :  True : NonNegativeReals
            CauBong wind :    0.0 :  None : 4294.23072802 : False :  True : NonNegativeReals
     ChonThanh bioenergy

In [21]:
g = n.model.generator_p_nom.values()

In [24]:
v = g.send(None)

In [37]:
print(O)

None
